In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#QPU connection
try:
    from qat.qlmaas import QLMaaSConnection
    connection = QLMaaSConnection(hostname="qlm")#, port=...)
    LinAlg = connection.get_qpu("qat.qpus:LinAlg")
    lineal_qpu = LinAlg()
except ImportError:
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()

## 1. Quantum Multiplexors

Implementation of loading data routines using the *Lov Grover and Terry Rudolph* routines directly, using controlled rotations by state, is highly inneficient. In general the use of controlled rotations generate highly deep quantum circuits prone to errors. 
A more efficient approach is the use of Quantum Multiplexors where the only controlled gates are c-Not ones. 

The operation we want to implement is a controlled rotation on an angle $\theta_i$ controlled by a state $|i\rangle$. 

For a 2 qbits state the operator should be:

$$
\hat{U}|q_0q_1\rangle= \{ \begin{array}{ll}
      |0\rangle\hat{R}_y(\theta_0)|q_1\rangle  & if \;|q_0\rangle = |0\rangle \\
      |1\rangle\hat{R}_y(\theta_1)|q_1\rangle  & if \;|q_0\rangle = |1\rangle \\
\end{array}
$$

With a quantum multiplexor this operation can be implemented as:

$$\hat{U}(\theta_0, \theta_1)|q_0q_1\rangle= \left( \mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0+\theta_1}{2}) \right) \hat{C}_{NOT} \left(\mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0-\theta_1}{2}) \right) \hat{C}_{NOT}|q_0q_1\rangle$$

The circuit representation for this operator is:

![alt text](./QM_01.png)

As can be seen the quantum multiplexor needs only $C_{NOT}$ gates that are implemented in easier way than controlled rotations


For controlled state rotations of more angles quantum multiplexors can be implemented in a recursively way. For example if we want apply a controlled 4 angle rotation $[\theta_0, \theta_1, \theta_2, \theta_3]$ we can use following circuit:

![alt text](./QM_02.png)


In general a quantum multiplexor with $m$ control qubits can be decomposed as 2 multiplexors with $m − 1$ control qubits and 2 $\hat{C}_{NOT}$ gates:
![alt text](./QM_03.png)

In the **QuantumMultiplexors_Module** there are an implementation for a quantum multiplexor controlled by $m$ qbits and the functions neede to use them in order to load probability and a Integral function

## 2. QuantumMultiplexors_Module

In order to use this module the basic functions are:
1. LoadProbability_Gate: Loads a probability distribution in a quantum state using multiplexors: so this function creates $\mathcal{P}$ gate
2. LoadIntegralFunction_Gate: Loads the integral of a function in a quantum state using multiplexors, so this function creates $\mathcal{R}$ gate

In order to do the data loading we need to discretizate it so:

In [ ]:
from AuxiliarFunctions import  get_histogram, PostProcessResults, TestBins
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

### 2.1 LoadProbability_Gate

The $\mathcal{P}$ is implemented by an Abstract qlm gate called **LoadProbability_Gate**. In order to create the gate only the numpy arrays with the probability should be provided.

The following function,**LoadProbabilityProgram**, creates a Quantum Program for loading a given probability numpy array on a quantum Circuit using the aforementioned gate.

In [ ]:
def LoadProbabilityProgram(p_X):
    """
    Creates a Quantum Program for loading an input numpy array with a probability distribution.
    Inputs:
        * p_X: np.array. Probability distribution of size m. Mandatory: m=2^n where n is the number qbits of the quantum circuit. 
    Outputs:
        * qprog: qlm program for loading input probability
    """
    #Qbits of the Quantum circuit depends on Probability length
    nqbits = TestBins(p_X, 'Probability')
    
    from qat.lang.AQASM import Program
    qprog = Program()
    qbits = qprog.qalloc(nqbits)
    #Creation of P_gate
    from QuantumMultiplexors_Module import LoadProbability_Gate
    P_gate = LoadProbability_Gate(p_X)
    #Apply Abstract gate to the qbits
    qprog.apply(P_gate, qbits)
    return qprog


In [ ]:
circuit_P = LoadProbabilityProgram(p_X).to_circ()

In [ ]:
%qatdisplay circuit_P --depth 0

Following cell simulates the circuit an postprocces result in order to test in probability was properly loaded. The results of the simulation is given to the **PostProcessResults** which create a pandas dataframe with the posible states and teh correspondings probabilities and amplitudes.

In [ ]:
jobP = circuit_P.to_job()
resultP = lineal_qpu.submit(jobP)
P_results = PostProcessResults(resultP.join())


In [ ]:
P_results.head()

In order to check if the operation was done properly we can compare the outpus probability of each state and the probabilities wanted to load:

In [ ]:
np.isclose(P_results['Probability'], p_X).all()

### 2.2 LoadIntegralFunction_Gate

The $\mathcal{R}$ is implemented by an Abstract qlm gate called **LoadIntegralFunction_Gate**. For creating this gate a numpy array with the function evaluation shoul be provided. The length of this array should be: $m=2^n$ where n is an integer. The created gate is a $n+1$ gate where the last qbit codifies the integral of the function.

Following **LoadIntegralProgram** function creates a complete Quantum Program for loading Integral of discretized function $f(x)$ using the programed $\mathcal{R}$ gate. The size of the function array should be $2^n$ and the circuit will have $n+1$ qbit where the last one qbit will enconde the integral of the $f(x)$

In [ ]:
def LoadIntegralProgram(f_X):
    """
    Creates a Quantum Circuit for loading the integral of the input numpy array with a function evaluation 
    Inputs:
        * f_X: np.array. Function evaluation of size m. Mandatory: m=2^n where n is the number
        qbits of the quantum circuit. 
    Outputs:
        * program: qlm program for loading integral of the input function
    """
    #Qbits of the Quantum circuit depends on Function array length
    nqbits = TestBins(f_X, 'Function')
    
    from qat.lang.AQASM import Program, H
    qprog = Program()
    #The additional qbit is where the integral will be encoded
    qbits = qprog.qalloc(nqbits+1)
    for i in range(nqbits):
        qprog.apply(H, qbits[i])
    #Creation of P_gate
    from QuantumMultiplexors_Module import LoadIntegralFunction_Gate
    R_gate = LoadIntegralFunction_Gate(f_X)
    #Apply Abstract gate to the qbits
    qprog.apply(R_gate, qbits)
    return qprog


In [ ]:
circuit_R = LoadIntegralProgram(f_X).to_circ()

In [ ]:
%qatdisplay circuit_R --depth 0

In this case we are only interested in the last qbit of the circuit that encodes the desIred integral. Following cell simulates the circuit from *LoadIntegralProgram* and post process the results using *PostProcessResults* function which ouputs a pandas dataframe with the posible states (measurement of the last qbit) and the corresponding probability

In [ ]:
jobR = circuit_R.to_job(qubits = [n_qbits])
result = lineal_qpu.submit(jobR)
R_results = PostProcessResults(result.join())

In [ ]:
#Integral of f(x)
MeasurementIntegral = R_results['Probability'][1]*2**(n_qbits)
np.isclose(MeasurementIntegral, sum(f_X))

### 2.3 Load Complete Data

Now we are going to use $\mathcal{P}$ and $\mathcal{R}$ implemented gates in order to load $E_{x\sim p}(f)$ in a quntum circuit. 

$$E_{x\sim p}(f) = \sum_{x=0}^{2^n-1} p(x)f(x)dx$$

In [ ]:
def LoadingData(p_X, f_X):
    """
    Load all the mandatory data to load in a quantum program the expected value 
    of a function f(x) over a x following a probability distribution p(x).
    Inputs:
        * p_X: np.array. Array of the discretized probability density
        * f_X: np.array. Array of the discretized funcion
    Outpus:
        * qprog: quantum program for loading the expected value of f(x) for x following a p(x) distribution
    """
    #Testing input
    nqbits_p = TestBins(p_X, 'Probability')
    nqbits_f = TestBins(f_X, 'Function')
    assert nqbits_p == nqbits_f, 'Arrays lenght are not equal!!'
    nqbits = nqbits_p
    
    #Creation of Gates
    from QuantumMultiplexors_Module import LoadProbability_Gate
    P_gate = LoadProbability_Gate(p_X)
    from QuantumMultiplexors_Module import LoadIntegralFunction_Gate
    R_gate = LoadIntegralFunction_Gate(f_X)
    
    from qat.lang.AQASM import Program
    qprog = Program()
    qbits = qprog.qalloc(nqbits+1)
    #Load Probability
    qprog.apply(P_gate, qbits[:-1])
    #Load integral on the last qbit
    qprog.apply(R_gate, qbits)
    return qprog

In [ ]:
circuitPR = LoadingData(p_X, f_X).to_circ()

In [ ]:
%qatdisplay circuitPR --depth 0

In [ ]:
jobPR = circuitPR.to_job(qubits = [n_qbits])
resultPR = lineal_qpu.submit(jobPR)
PR_results = PostProcessResults(resultPR.join())

As explained before the $E_{x\sim p}(f)$ will be loaded in the $|1\rangle$ state of the last qbit of the circuit

In [ ]:
#Integral of f(x)
MeasurementIntegral = PR_results['Probability'][1]

In [ ]:
np.isclose(MeasurementIntegral, sum(p_X*f_X))

## 3 Sumary and Important notes

In **01_Dataloading_Module_Use** Notebook the *dataloading_module* and *dataloading_module_examples* were explained and used. 
Present Notebook try to mimic the **01_Dataloading_Module_Use** one but using and explained **QuantumMultiplexors_Module** and **QuantumMultiplexors_Module_examples**. 

The functions and gates implemented in *dataloading_module* and in the **QuantumMultiplexors_Module** have the same functionality: **load data, probability functions and integral functions** in a quantum state but the implementation was changed:
* In the *dataloading_module*: several controlled by states rotations were straightoforward implemented in order to create the gates to load the data.
* In the **QuantumMultiplexors_Module**: the loading data gates were implemented in a much more efficient way using **quantum multiplexors** where the controlloed rotations are subsituted by simple (and intelligent) qbit Rotations and $\hat{C}_{NOT}$ gates

## 4. File Scripts

All the functions generated in this Notebook were stored in a python file, **QuantumMultiplexors_Module_examples.py** in order to use it in an easy way:


In [ ]:
from AuxiliarFunctions import get_histogram, PostProcessResults
def p(x):
    return x*x
def f(x):
    return np.sin(x)

#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 
X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
from QuantumMultiplexors_Module_examples import LoadProbabilityProgram, LoadingData, LoadIntegralProgram

In [ ]:
circuitP = LoadProbabilityProgram(p_X).to_circ()
circuitF = LoadIntegralProgram(f_X).to_circ()
circuitPF = LoadingData(p_X, f_X).to_circ()

### 4.1 Probability Load

In [ ]:
%qatdisplay circuitP

In [ ]:
jobP = circuitP.to_job()
resultP = lineal_qpu.submit(jobP)
P_results = PostProcessResults(resultP.join())
np.isclose(P_results['Probability'], p_X).all()

### 4.2 Integral Load

In [ ]:
%qatdisplay circuitF

In [ ]:
jobF = circuitF.to_job(qubits = [n_qbits])
resultF = lineal_qpu.submit(jobF)
F_results = PostProcessResults(resultF.join())
MeasurementIntegral = F_results['Probability'][1]*2**(n_qbits)
np.isclose(MeasurementIntegral, sum(f_X))

### 4.3 Complete Load

In [ ]:
%qatdisplay circuitPF

In [ ]:
jobPF = circuitPF.to_job(qubits = [n_qbits])
resultPF = lineal_qpu.submit(jobPF)
PF_results = PostProcessResults(resultPF.join())
MeasurementIntegral = PF_results['Probability'][1]
np.isclose(MeasurementIntegral, sum(p_X*f_X))